In [23]:
#Import libraries

import numpy as np
import math as math
import cmath as cmath
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.stats import norm

In [24]:
#Class to hold the relevant functions

class Heston(object):
    
    def __init__(self,S0,K,tau,r,kappa,theta,v0,lamda,sigma,rho): #Constructor for initiating the class
        
        self.x0=math.log(S0)
        self.ln_k=math.log(K)
        self.r=r
        self.v0=v0
        self.kappa=kappa
        self.theta=theta
        self.lamda=lamda
        self.sigma=sigma
        self.rho=rho
        self.tau=tau
   
        self.a=kappa*theta
        self.u=[0.5,-0.5]
        self.b=[kappa+lamda-rho*sigma,kappa+lamda]
        
    def reset_parameters(self,S0,K,tau,r,kappa,theta,v0,lamda,sigma,rho): # Function for resetting the constant parameters
        self.x0=math.log(S0)
        self.ln_k=math.log(K)
        self.r=r
        self.v0=v0
        self.kappa=kappa
        self.theta=theta
        self.lamda=lamda
        self.sigma=sigma
        self.rho=rho
        self.tau=tau
   
        self.a=kappa*theta
        self.u=[0.5,-0.5]
        self.b=[kappa+lamda-rho*sigma,kappa+lamda]       
       
 
    def f1(self,phi):#f1 only using a copy of the previous code with minimal change, i.e.,j=0 replaes loop
        
        d=[0.0,0.0]
        g=[0.0,0.0]
        C=[0.0,0.0]
        D=[0.0,0.0]
        edt=[0.0,0.0]
        gedt=[0.0,0.0]
        f=[0.0,0.0]

        j=0

        temp=self.b[j]-1j*self.rho*self.sigma*phi

        d[j]=cmath.sqrt(temp**2-self.sigma**2*(2.0*self.u[j]*phi*1j-phi**2))
        g[j]=(temp+d[j])/(temp-d[j])

        edt[j]=cmath.exp(d[j]*self.tau)
        gedt[j]=1.0-g[j]*edt[j]

        D[j]=(temp+d[j])*(1.0-edt[j])/gedt[j]/self.sigma/self.sigma
        C[j]=self.r*phi*self.tau*1j+self.a/self.sigma/self.sigma*((temp+d[j])*self.tau-2.0*cmath.log(gedt[j]/(1.0-g[j])))
        f[j]=cmath.exp(C[j]+D[j]*self.v0+1j*phi*self.x0)

        return f[0]    
    
    def f2(self,phi):# f2 only using a copy of the previous code with minimal change, i.e.,now j=1 replaes loop
        
        d=[0.0,0.0]
        g=[0.0,0.0]
        C=[0.0,0.0]
        D=[0.0,0.0]
        edt=[0.0,0.0]
        gedt=[0.0,0.0]
        f=[0.0,0.0]
        
        j=1

        temp=self.b[j]-1j*self.rho*self.sigma*phi

        d[j]=cmath.sqrt(temp**2-self.sigma**2*(2.0*self.u[j]*phi*1j-phi**2))
        g[j]=(temp+d[j])/(temp-d[j])

        edt[j]=cmath.exp(d[j]*self.tau)
        gedt[j]=1.0-g[j]*edt[j]

        D[j]=(temp+d[j])*(1.0-edt[j])/gedt[j]/self.sigma/self.sigma
        C[j]=self.r*phi*self.tau*1j+self.a/self.sigma/self.sigma*((temp+d[j])*self.tau-2.0*cmath.log(gedt[j]/(1.0-g[j])))
        f[j]=cmath.exp(C[j]+D[j]*self.v0+1j*phi*self.x0)
        
        return f[1]     
      
    def P1_integrand(self,phi): #Returns the integrand  that appears in the P1 formula
        temp=cmath.exp(-1j*phi*self.ln_k)*self.f1(phi)/1j/phi
        return temp.real

    def P2_integrand(self,phi):  #Returns the integrand  that appears in the P1 formula
        temp=cmath.exp(-1j*phi*self.ln_k)*self.f2(phi)/1j/phi
        return temp.real
    
    def Probabilities(self):
        pi_i=1.0/math.pi
        P1=0.5+pi_i*quad(self.P1_integrand,0,100)[0] # The b values for integration are sometimes finicky
        P2=0.5+pi_i*quad(self.P2_integrand,0,100)[0] # Try infinity, if not 100 is usually good enough
        P=[P1,P2]
        return P
    
    def price(self):
        Ps=self.Probabilities()
        
        call_price=math.exp(self.x0)*Ps[0]-math.exp(self.ln_k-self.r*self.tau)*Ps[1]
        put_price=call_price-(math.exp(self.x0)-math.exp(self.ln_k-self.r*self.tau))
        
        output={
            "Call price":call_price,
            "Put price":put_price
        }
        return output
    
        
        


In [25]:
class BlackScholes(object):
    def __init__(self,S0,K,T,r,sigma): #Constructor for initiating the class
        
        self.S0=S0
        self.K=K
        self.r=r
        self.T = T
        self.sigma=sigma
   
    def reset_parameters(self,S0,K,T,r,sigma): # Function for resetting the constant parameters
        self.S0=S0
        self.K=K
        self.r=r
        self.T = T
        self.sigma=sigma

    def price(self):
        d1 = (np.log(self.S0/self.K) + (self.r + self.sigma**2/2)*self.T)/(self.sigma*np.sqrt(self.T))
        d2 = d1 - self.sigma*np.sqrt(self.T)
        call_price = self.S0*norm.cdf(d1, 0, 1) - self.K*np.exp(-self.r*self.T)*norm.cdf(d2, 0, 1)
        put_price = self.K*np.exp(-self.r*self.T)*norm.cdf(-d2, 0, 1) - self.S0*norm.cdf(-d1, 0, 1)
        
        output={
            "Call price":call_price,
            "Put price":put_price
        }
        return output

In [26]:
class BAPM(object):
    def __init__(self, S0, K, T, r, N, u):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.N = N
        self.u = u
        self.d = 1/u # to ensure recombining tree
    
    def reset_params(self, S0, K, T, r, N, u):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.N = N
        self.u = u
        self.d = 1/u

    def price(self):
        dt = self.T/self.N
        q = (np.exp(self.r*dt) - self.d) / (self.u-self.d)
        disc = np.exp(-self.r*dt)

        S = np.zeros(self.N+1)
        S[0] = self.S0*self.d**self.N
        for j in range(1,self.N+1):
            S[j] = S[j-1]*self.u/self.d

        C = np.zeros(self.N+1)
        for i in range(0,self.N+1):
            C[i] = max(0, S[i] - self.K)

        for i in np.arange(self.N,0,-1):
            for j in range(0,i):
                C[j] = disc * ( q*C[j+1] + (1-q)*C[j] )

        P = np.zeros(self.N+1)
        for i in range(0,self.N+1):
            P[i] = max(0, self.K - S[i])

        for i in np.arange(self.N,0,-1):
            for j in range(0,i):
                P[j] = disc * ( q*P[j+1] + (1-q)*P[j] )
        
        output={
            "Call price":C[0],
            "Put price":P[0]
        }
        return output
        